In [1]:
import torch
import utils
import os
import numpy as np
from collections import OrderedDict
from vgg_face_dag import vgg_face_dag
from ami_model import AmIModel

device = torch.device('cuda')
# vgg_weight = './vgg_face_caffe.pth'
# vgg_weight = './keras_vgg_face.pth'
vgg_weight = './vgg_face_dag.pth'

# ami model
SKIP_LAYERS = utils.SKIP_LAYERS
attribute_model = AmIModel(vgg_face_dag(vgg_weight), 60., 2.15, 50.)
attribute_model.to(device)
attribute_model.eval()
attribute_model.register_my_hook(skip_layers=SKIP_LAYERS, ami_data=utils.load_neuron_set_lists())

# original model
vgg_net = vgg_face_dag(vgg_weight)
vgg_net.to(device)
vgg_net.eval()

register hook for relu1_1
register hook for relu1_2
register hook for pool1
register hook for relu2_1
register hook for relu2_2
register hook for pool2
register hook for relu3_1
register hook for relu3_2
register hook for relu3_3
register hook for pool3
register hook for relu4_1
register hook for relu4_2
register hook for relu4_3
register hook for pool4
register hook for relu5_1
register hook for relu5_2
register hook for relu5_3
register hook for pool5
register hook for relu6
register hook for relu7


Vgg_face_dag(
  (conv1_1): Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu1_1): ReLU(inplace=True)
  (conv1_2): Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu1_2): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu2_1): ReLU(inplace=True)
  (conv2_2): Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu2_2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
  (conv3_1): Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_1): ReLU(inplace=True)
  (conv3_2): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_2): ReLU(inplace=True)
  (conv3_3): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_3): ReLU(inplace=True)
  (pool3)

In [2]:
vgg_root   = '../../data/vgg_face_caffe/'
vgg_names  = utils.read_list(vgg_root + 'names.txt')

In [3]:
img_dir = '/tmp/sized_full_images/'
img_list = utils.read_list('./benign_list.txt')

img_count = 0
adv_count = 0

with torch.no_grad():
    for img_name in img_list:
        img_name  = img_name.strip()
        img_path  = os.path.join(img_dir, img_name)

        prob_original  = vgg_net(utils.get_data(img_path).to(device))
        prob_attribute = attribute_model(utils.get_data(img_path).to(device))
        id_original    = torch.max(prob_original, 1)[1].item()
        id_attribute   = torch.max(prob_attribute, 1)[1].item()

        id_gold = utils.get_identity(img_name, vgg_names)
        
        if id_gold == id_original:
            img_count += 1
        
            ### consistency observer for detecting adversarial samples ###
            if id_original != id_attribute:
                adv_count += 1

            # print('{:3} Gold-Original-Attribute: {:4}-{:4}-{:4}  |  Adversary: {:.4f} [{:3}/{:3}]'
            #       .format(img_count, id_gold, id_original, id_attribute, 
            #               1.0*adv_count/img_count, adv_count, img_count))
print(f'False positive rate: {adv_count}/{img_count}={adv_count/img_count}')

False positive rate: 33/343=0.09620991253644315
